In [1]:
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy.io as sio
import skimage.io
import h5py

import sys
sys.path.append('../Metric/')
sys.path.append('../../Visualization/')
sys.path.append('../../Data_Preprocessing/')
from Metric import *
from Data_Extractor import *
from Bilinear_Kernel import *

/home/admin-u6142160/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Load & Reorder:

In [2]:
# Load
path_raw_image = "../../Data/090085/All_Data/090085_20170531.h5"
path_road_mask = "../../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/motor_trunk_pri_sec_tert_uncl_track.tif"
path_topleft_coordinate = "../../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/posneg_topleft_coordinate_8"

raw_image = np.array(h5py.File(path_raw_image)['scene'])
road_mask = skimage.io.imread(path_road_mask)

data = h5py.File(path_topleft_coordinate, 'r')

pos_topleft_coord = np.array(data['positive_example'])
neg_topleft_coord = np.array(data['negative_example'])
data.close()

print(raw_image.shape, road_mask.shape)
print(raw_image.min())

(7, 7961, 8091) (7961, 8091)
-9999


In [3]:
# Construct training & test set
pos_size = pos_topleft_coord.shape[0]
neg_size = neg_topleft_coord.shape[0]

np.random.shuffle(pos_topleft_coord)
np.random.shuffle(neg_topleft_coord)

Train_Data = Data_Extractor (raw_image, road_mask, 8,
                             pos_topleft_coord = pos_topleft_coord[:int(0.75*pos_size) ,:],
                             neg_topleft_coord = neg_topleft_coord[:int(0.75*neg_size) ,:])

Test_Data  = Data_Extractor (raw_image, road_mask, 8,
                             pos_topleft_coord = pos_topleft_coord[ int(0.75*pos_size):int(0.90*pos_size),:],
                             neg_topleft_coord = neg_topleft_coord[ int(0.75*neg_size):int(0.90*neg_size),:])

print(Train_Data.size, Test_Data.size)

19131279 3826257


Model parameters: 

In [54]:
size = 128
band = 7

# Hyper parameters
conv_out = [0, 32, 64, 96]

class_output = 2 # number of possible classifications for the problem

use_weight = True
keep_rate = 0.5 # need regularization => otherwise NaN appears inside CNN

# class_weight = [Train_Data.pos_size/Train_Data.size, Train_Data.neg_size/Train_Data.size]

batch_size = 64
learning_rate = 9e-6
epoch = 15

# print(class_weight, '[neg, pos]')

Convolutional Layers:

In [71]:
tf.reset_default_graph()

with tf.variable_scope('input'):
    x = tf.placeholder(tf.float32, shape=[batch_size, size, size, band], name='x')
    y = tf.placeholder(tf.float32, shape=[batch_size, size, size, class_output], name='y')

    weight = tf.placeholder(tf.float32, shape=[batch_size, size, size, class_output], name='class_weight')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob') # dropout
    is_training = tf.placeholder(tf.bool, name='is_training') # batch norm


with tf.variable_scope('down_sampling'):
    # Convolutional Layer 1
    net = tf.contrib.layers.conv2d(inputs=x, num_outputs=conv_out[1], kernel_size=3, 
                                   stride=1, padding='SAME', scope='conv1')

    net = tf.contrib.layers.max_pool2d(inputs=net, kernel_size=2, stride=2, padding='VALID', scope='pool1')
    pool_1 = net
    
    # Convolutional Layer 2
    net = tf.contrib.layers.conv2d(inputs=net, num_outputs=conv_out[2], kernel_size=3, 
                                   stride=1, padding='SAME', scope='conv2')

    net = tf.contrib.layers.max_pool2d(inputs=net, kernel_size=2, stride=2, padding='VALID', scope='pool2')
    pool_2 = net
    
    # Convolutional Layer 3
    net = tf.contrib.layers.conv2d(inputs=net, num_outputs=conv_out[3], kernel_size=3, 
                                   stride=1, padding='SAME', scope='conv3')

    net = tf.contrib.layers.max_pool2d(inputs=net, kernel_size=2, stride=2, padding='VALID', scope='pool3')


with tf.variable_scope('up_sampling'):
    kernel_size = get_kernel_size(2)
    net = tf.contrib.layers.conv2d_transpose(inputs=net, num_outputs=conv_out[2], kernel_size=kernel_size, stride=2, 
                                             weights_initializer=tf.constant_initializer(get_bilinear_upsample_weights(2, conv_out[3], conv_out[2])), 
                                             scope='conv3_T')
    with tf.variable_scope('fuse_with_2'):
        net = net + pool_2

    net = tf.contrib.layers.conv2d_transpose(inputs=net, num_outputs=conv_out[1], kernel_size=kernel_size, stride=2, 
                                             weights_initializer=tf.constant_initializer(get_bilinear_upsample_weights(2, conv_out[2], conv_out[1])), 
                                             scope='conv2_T')
    with tf.variable_scope('fuse_with_1'):
        net = net + pool_1
    
    net = tf.contrib.layers.conv2d_transpose(inputs=net, num_outputs=class_output, kernel_size=kernel_size, stride=2, 
                                             weights_initializer=tf.constant_initializer(get_bilinear_upsample_weights(2, conv_out[1], class_output)), 
                                             scope='conv1_T')
    pred = tf.nn.softmax(net)
#     with tf.variable_scope('fuse_with_mean_input'):
#         net = net + tf.reduce_mean(x, axis=-1)

/home/admin-u6142160/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


Cost function & optimizer:

In [73]:
logits = tf.reshape(net, (-1, class_output))
labels = tf.to_float(tf.reshape(y, (-1, class_output)))

softmax = tf.nn.softmax(logits) + tf.constant(value=1e-7) # because of the numerical instableness

if use_weight:
    weight = tf.reshape(weight,(-1, class_output))
    cross_entropy = -tf.reduce_sum(tf.multiply(labels * tf.log(softmax), weight) )
else:
    cross_entropy = -tf.reduce_sum(labels * tf.log(softmax), reduction_indices=[1])

mean_cross_entropy = tf.reduce_mean(cross_entropy, name='mean_cross_entropy')

# Ensures that we execute the update_ops before performing the train_step
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

/home/admin-u6142160/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/admin-u6142160/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/admin-u6142160/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/admin-u6142160/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.sign

Train & monitor:

In [ ]:
saver = tf.train.Saver()

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

learning_curve = []
iteration = int(Train_Data.size/batch_size) + 1

for epoch_num in range(epoch):
    for iter_num in range(iteration):
        
        batch_x, batch_y = Train_Data.get_patches(batch_size=64)
        batch_x = batch_x.transpose((0, 2, 3, 1))
        train_step.run(feed_dict={x: batch_x, y: batch_y, keep_prob: keep_rate, is_training: True})

    # snap shot
    cur_cost = cross_entropy.eval(feed_dict={x:batch_x, y: batch_y, keep_prob: 1, is_training:False})
    print("cross entropy = ", cur_cost)
    learning_curve.append(cur_cost)
print("finish")

In [ ]:
# plot training curve
plt.figsize=(9,5)
plt.plot(learning_curve)
plt.title('learning_curve')
plt.show()

## (Optional) Save the model

In [ ]:
save_path = './Result/motor_trunk_pri_sec_tert/CNN - central pixel/'
model_name = 'motor_trunk_pri_sec_tert'
saver.save(sess, save_path + model_name)

h5f = h5py.File(save_path + "training_info.h5", 'w')

h5f_Index = h5f.create_group("Index")

h5f_Index_Train = h5f_Index.create_group(name='Train')
h5f_Index_Train.create_dataset (name='pos', data=Train_Data.pos_topleft_coord)
h5f_Index_Train.create_dataset (name='neg', data=Train_Data.neg_topleft_coord)

h5f_Index_Test  = h5f_Index.create_group(name='Test')
h5f_Index_Test.create_dataset (name='pos', data=Test_Data.pos_topleft_coord)
h5f_Index_Test.create_dataset (name='neg', data=Test_Data.neg_topleft_coord)

h5f_Norm = h5f.create_group("Norm")
h5f_Norm.create_dataset(name='mu', shape=mu.shape, data=mu)
h5f_Norm.create_dataset(name='sigma', shape=sigma.shape, data=sigma)
h5f.close()

Evaluate:

In [ ]:
train_metric = Metric()

for img, mask in Train_Data.iterate_data():
    batch = [((img-mu)/sigma).reshape((-1,band,height,width)).transpose((0,2,3,1)).astype(np.float32), 
             np.matrix(mask).astype(int).T]

    # record metric
    pred = prediction.eval(feed_dict={x:batch[0], keep_prob: 1, is_training: False})
    train_metric.accumulate(pred, batch[1])
    
train_metric.print_info()

In [ ]:
test_metric = Metric()

for img, mask in Test_Data.iterate_data():
    batch = [((img-mu)/sigma).reshape((-1,band,height,width)).transpose((0,2,3,1)).astype(np.float32), 
             np.matrix(mask).astype(int).T]
    
    # record metric
    pred = prediction.eval(feed_dict={x:batch[0], keep_prob: 1, is_training: False})
    test_metric.accumulate(pred, batch[1])
    
test_metric.print_info()